# Movies Recommender System

Importing libraries:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

from nltk.stem.snowball import SnowballStemmer

from surprise import Reader
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate, KFold

import warnings; warnings.simplefilter('ignore')

## Data collection and cleaning
Reading and Droping unwanted data:


In [2]:
md = pd. read_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/movies_metadata.csv')
md[['genres']]
md = md.drop(['budget', 'homepage', 'video', 'revenue', 'runtime', 'status'], axis=1)
md
md = md.drop(['poster_path','original_title'], axis=1)
md = md.drop(['spoken_languages'], axis=1)
#md
md.shape

(45466, 15)

In [3]:
#cleaning genres into neat list
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
#md[['genres']]

In [4]:
#further cleaning few columns
md_clean = md[md['belongs_to_collection'].notnull()]
md_clean['belongs_to_collection'] = md_clean['belongs_to_collection'].apply(literal_eval).apply(lambda x: x['name'] if isinstance(x, dict) else np.nan)
md_clean = md_clean[md_clean['belongs_to_collection'].notnull()]
md_clean

md['production_countries'] = md['production_countries'].fillna('[]').apply(literal_eval)
md_clean['production_countries'] = md['production_countries'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

md['production_companies'] = md['production_companies'].fillna('[]').apply(literal_eval)
md_clean['production_companies'] = md['production_companies'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

md_clean.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/belongs_to.csv', index=False)
#md_clean.head(10)

handling genre:

In [5]:
#spliting through genres to get graphs  
#weighing genre (*3)
s = md_clean.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md_clean = md_clean.drop('genres', axis=1).join(s)
gen_md_clean.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/gen_md_clean_nowr.csv', index=False)

# s = renewed_scores.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
# s.name = 'genre'
# gen_md_clean = renewed_scores.drop('genres', axis=1).join(s)
# gen_md_clean.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/gen_md_clean.csv', index=False)

handling companies and producers:

In [6]:
com = md_clean.apply(lambda x: pd.Series(x['production_companies']),axis=1).stack().reset_index(level=1, drop=True)
com.name = 'companies'
company_md_clean = gen_md_clean.drop('production_companies', axis=1).join(com)
company_md_clean.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/company_md_clean_nowr.csv', index=False)


In [7]:
#com

In [8]:
#company_md_clean.head(15)

In [9]:
coun = md_clean.apply(lambda x: pd.Series(x['production_countries']),axis=1).stack().reset_index(level=1, drop=True)
coun.name = 'countries'
countries_md_clean = company_md_clean.drop('production_countries', axis=1).join(coun)
countries_md_clean.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/countries_md_clean.csv', index=False)


In [10]:
#countries_md_clean.head(15)

# Content Based Recommender


In [11]:
links_small = pd.read_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [12]:
md = md.drop([19730, 29503, 35587])

In [13]:
md['id'] = md['id'].astype('int')

In [14]:
#smaller version of database
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 15)

In [15]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')
#smd[['description']]

smd['description'] = smd['description'].apply(lambda x: [stemmer.stem(i) for i in x])

In [16]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [17]:
'MeemxszfsUHNJK'.lower()

'meemxszfsuhnjk'

In [18]:
credits = pd.read_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/credits.csv')
keywords = pd.read_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/keywords.csv')

In [19]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [20]:
md.shape

(45463, 15)

In [21]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

director and actor eda:

In [22]:
#ok_start
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 18)

In [23]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [24]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [25]:
smd['director'] = smd['crew'].apply(get_director)

In [26]:
#smd[['director']]

In [27]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [28]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [29]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
#smd.head(12)

In [30]:
smd['director'] = smd['director'].astype('str').apply(lambda y: str.lower(y.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda y: [y,y,y])
smd.head(12)
smd.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/castCrew.csv', index=False)
#md_clean.head(10)
#smd.head(12)

In [31]:
s = smd.apply(lambda x: pd.Series(x['cast']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'cast'
cast_clean = smd.drop('cast', axis=1).join(s)
cast_clean.to_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/new created datasets/cast_clean_nowr.csv', index=False)

#

#### Keywords

In [32]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s = s[s > 1]

In [33]:
# stemmer = SnowballStemmer('english')
# stemmer.stem('dogs')

In [34]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [35]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
#smd[['keywords']]

In [36]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')
#smd[['description']]

In [69]:
smd['khichdi'] = smd['keywords'] + smd['cast']*3 + smd['director']*3 + smd['genres'] 
smd['khichdi'] = smd['khichdi'].apply(lambda x: ' '.join(x))
#smd[['khichdi']]

In [38]:
#smd['khichdi'] = smd['tagline']+ smd['khichdi'] 
#smd[['khichdi']]

In [39]:
len(smd)

9219

In [40]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['khichdi'])
count_matrix.shape

(9219, 139772)

In [41]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [42]:
#this be my code too:
def reco_cosine(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [43]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [70]:
reco_cosine('Resident Evil').head(10)

8212    Resident Evil: Retribution
7689      Resident Evil: Afterlife
90                        Shopping
5609       AVP: Alien vs. Predator
7094                    Death Race
7990          The Three Musketeers
1854                       Soldier
42                   Mortal Kombat
1279                 Event Horizon
5636     Resident Evil: Apocalypse
Name: title, dtype: object

In [71]:
reco_cosine('Death Race').head(10)

90                        Shopping
5609       AVP: Alien vs. Predator
7990          The Three Musketeers
7689      Resident Evil: Afterlife
4030                 Resident Evil
1854                       Soldier
8212    Resident Evil: Retribution
42                   Mortal Kombat
1279                 Event Horizon
6905                   I Am Legend
Name: title, dtype: object

In [46]:
def wr_reco(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    wr_tunning = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    wr_tunning['vote_count'] = wr_tunning['vote_count'].astype('int')
    wr_tunning['vote_average'] = wr_tunning['vote_average'].astype('int')
    
    v = wr_tunning['vote_count']
    R = wr_tunning['vote_average']
    wr_tunning['wr'] = (v/(v+m) * R) + (m/(m+v) * C)
    wr_tunning = wr_tunning.sort_values('wr', ascending=False).head(10)
    return wr_tunning

In [47]:
wr_reco('Mean Girls')

,title,vote_count,vote_average,wr
1547,The Breakfast Club,2189,7,6.766751
8883,The DUFF,1372,6,5.883347
3712,The Princess Diaries,1063,6,5.858655
4763,Freaky Friday,919,6,5.843186
6277,Just Like Heaven,595,6,5.791958
6959,The Spiderwick Chronicles,593,6,5.791538
2005,She's All That,425,5,5.231764
7494,American Pie Presents: The Book of Love,454,5,5.223865
7332,Ghosts of Girlfriends Past,716,5,5.171157
7905,Mr. Popper's Penguins,775,5,5.162540


## Collaborative Filtering


In [48]:
reader = Reader()

In [49]:
ratings = pd.read_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [50]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
#data.split(n_folds=5)
kf = KFold(n_splits=50)
kf.split(data)

<generator object KFold.split at 0x000001396AD3D510>

In [51]:
svd = SVD()
cv2 = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)
cv2

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8844  0.8893  0.8849  0.9004  0.8909  0.9022  0.9011  0.8984  0.8810  0.8839  0.8917  0.0078  
MAE (testset)     0.6850  0.6800  0.6803  0.6935  0.6839  0.6940  0.6908  0.6904  0.6773  0.6823  0.6857  0.0057  
Fit time          7.32    5.92    7.57    6.44    7.16    7.26    7.35    6.35    5.97    7.27    6.86    0.59    
Test time         0.08    0.07    0.08    0.10    0.11    0.09    0.09    0.09    0.11    0.12    0.09    0.02    


{'test_rmse': array([0.88438702, 0.88926472, 0.88489238, 0.90035588, 0.89091289,
        0.90224466, 0.90110672, 0.89844221, 0.8810141 , 0.88389904]),
 'test_mae': array([0.68503236, 0.68000752, 0.68027921, 0.69346249, 0.6839113 ,
        0.69404765, 0.6907721 , 0.69038622, 0.67725933, 0.68233152]),
 'fit_time': (7.320825099945068,
  5.9246039390563965,
  7.57370138168335,
  6.4433252811431885,
  7.162984132766724,
  7.258825302124023,
  7.354800224304199,
  6.3493452072143555,
  5.971561908721924,
  7.270015716552734),
 'test_time': (0.07797408103942871,
  0.07497525215148926,
  0.07595586776733398,
  0.0989232063293457,
  0.11295509338378906,
  0.09096670150756836,
  0.08696794509887695,
  0.09094810485839844,
  0.1099555492401123,
  0.11993241310119629)}

We get a mean **Root Mean Sqaure Error** of 0.8963 which is more than good enough for our case. Let us now train on our dataset and arrive at predictions.

In [52]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [53]:
testset = trainset.build_anti_testset()
predictions = svd.test(testset)


In [54]:
predictions

[Prediction(uid=1, iid=10, r_ui=3.543608255669773, est=2.4701771360891858, details={'was_impossible': False}),
 Prediction(uid=1, iid=17, r_ui=3.543608255669773, est=2.9500481179586204, details={'was_impossible': False}),
 Prediction(uid=1, iid=39, r_ui=3.543608255669773, est=2.7766616679989613, details={'was_impossible': False}),
 Prediction(uid=1, iid=47, r_ui=3.543608255669773, est=3.333158874311699, details={'was_impossible': False}),
 Prediction(uid=1, iid=50, r_ui=3.543608255669773, est=3.5101102697595357, details={'was_impossible': False}),
 Prediction(uid=1, iid=52, r_ui=3.543608255669773, est=2.6717685546398497, details={'was_impossible': False}),
 Prediction(uid=1, iid=62, r_ui=3.543608255669773, est=2.942655668756817, details={'was_impossible': False}),
 Prediction(uid=1, iid=110, r_ui=3.543608255669773, est=3.1590081385532156, details={'was_impossible': False}),
 Prediction(uid=1, iid=144, r_ui=3.543608255669773, est=2.769742460346241, details={'was_impossible': False}),
 P

Let us pick user 5000 and check the ratings s/he has given.

In [55]:
ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [56]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.770933797778776, details={'was_impossible': False})

In [57]:
for item in 
svd.predict(userId, indices_map.loc[x]['movieId']).est

SyntaxError: invalid syntax (<ipython-input-57-d3afa843e15f>, line 1)

For movie with ID 302, we get an estimated prediction of **2.686**. One startling feature of this recommender system is that it doesn't care what the movie is (or what it contains). It works purely on the basis of an assigned movie ID and tries to predict ratings based on how the other users have predicted the movie.

In [58]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [59]:
id_map = pd.read_csv('C:/Users/sunaina.rathod/Documents/Evaluation/Movie Reco/DataSet/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

id_map

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0
...,...,...
The Last Brickmaker in America,161944,159550.0
Rustom,162542,392572.0
Mohenjo Daro,162672,402672.0


In [60]:
estimaaate = []
for item in id_map['movieId'] :
    estimaaate.append(svd.predict('1', item).est)
estimaaate
id_map['est_svd'] = estimaaate
id_map = id_map.sort_values('est_svd', ascending=False)
id_map

md_new = id_map.merge(smd[['title', 'id', 'khichdi', 'vote_count', 'vote_average']], on='id')
md_new

,movieId,id,est_svd,title,khichdi,vote_count,vote_average
0,318,278.0,4.506329,The Shawshank Redemption,Fear can hold you prisoner. Hope can set you f...,8358.0,8.5
1,858,238.0,4.449344,The Godfather,An offer you can't refuse.itali loveatfirstsig...,6024.0,8.5
2,969,488.0,4.418848,The African Queen,The greatest adventure a man ever lived...with...,232.0,7.4
3,926,705.0,4.392309,All About Eve,It's all about women... and their men!jealousi...,367.0,8.0
4,2064,1779.0,4.376489,Roger & Me,capit econom unemploy michaelmoore rogerb.smit...,92.0,7.4
...,...,...,...,...,...,...,...
10202,2701,8487.0,2.335041,Wild Wild West,It's a whole new west.steampunk basedontvseri ...,1042.0,5.1
10203,181,9070.0,2.334643,Mighty Morphin Power Rangers: The Movie,The Power Is On!basedontvseri tokusatsu superh...,153.0,5.2
10204,546,9607.0,2.272658,Super Mario Bros.,This Ain't No Game.savingtheworld brotherbroth...,236.0,4.0
10205,1556,1639.0,2.182515,Speed 2: Cruise Control,"As the stakes get higher, the ride gets even f...",439.0,4.1


In [61]:
def hyb_reco(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = md_new.iloc[movie_indices][['title', 'vote_count', 'vote_average']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    wr_tunning = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    wr_tunning['vote_count'] = wr_tunning['vote_count'].astype('int')
    wr_tunning['vote_average'] = wr_tunning['vote_average'].astype('int')
    
    v = wr_tunning['vote_count']
    R = wr_tunning['vote_average']
    wr_tunning['wr'] = (v/(v+m) * R) + (m/(m+v) * C)
    wr_tunning = wr_tunning.sort_values('wr', ascending=False).head(10)
    return wr_tunning

In [62]:
hyb_reco('Resident Evil')

,title,vote_count,vote_average,wr
42,The Treasure of the Sierra Madre,285,7,6.512203
1973,Confessions,233,7,6.452966
7990,Die Hard 2,1920,6,5.981543
7094,Silent Hill,1093,6,5.969694
5636,Buried,853,6,5.962755
7875,Bean,602,6,5.951029
2059,District B13,572,6,5.949114
5609,Cry-Baby,289,6,5.919372
6736,Playing It Cool,258,5,5.344572
8428,xXx: State of the Union,565,4,4.462194


In [63]:
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        9214
Rustom                                                9215
Mohenjo Daro                                          9216
Shin Godzilla                                         9217
The Beatles: Eight Days a Week - The Touring Years    9218
Length: 9219, dtype: int64

In [64]:
indices_map = id_map.set_index('id')

In [65]:
def cf_reco(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    #sim_scores for the movie id
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    #svd on the generated similar content based movies list
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    #assign on the bases of est predicted score
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [66]:
#mycode
idx = indices['Resident Evil']
idxtmdbId = id_map.loc['Resident Evil']['id']
idxtmdbId

1576.0

In [67]:
sim_scores = list(enumerate(cosine_sim[int(idx)]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[1:26]
movie_indices = [i[0] for i in sim_scores]
movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]

In [68]:
cf_reco(1, 'Mean Girls')

,title,vote_count,vote_average,id,est
1547,The Breakfast Club,2189.0,7.8,2108,3.353600
4546,Pillow Talk,49.0,6.8,4952,2.912151
6449,Aquamarine,372.0,5.8,14191,2.886299
3956,Storytelling,64.0,6.4,16550,2.852131
5163,Just One of the Guys,64.0,6.4,24548,2.836171
6698,It's a Boy Girl Thing,279.0,6.3,37725,2.820467
7377,"I Love You, Beth Cooper",204.0,5.2,19840,2.786374
3273,Sugar & Spice,60.0,5.7,16723,2.784894
1704,Safe Men,17.0,6.3,16155,2.748923
7332,Ghosts of Girlfriends Past,716.0,5.6,12556,2.731681


We see that for our hybrid recommender, we get different recommendations for different users although the movie is the same. Hence, our recommendations are more personalized and tailored towards particular users.

## Conclusion

In this notebook, I have built 4 different recommendation engines based on different ideas and algorithms. They are as follows:

1. **Simple Recommender:** This system used overall TMDB Vote Count and Vote Averages to build Top Movies Charts, in general and for a specific genre. The IMDB Weighted Rating System was used to calculate ratings on which the sorting was finally performed.
2. **Content Based Recommender:** We built two content based engines; one that took movie overview and taglines as input and the other which took metadata such as cast, crew, genre and keywords to come up with predictions. We also deviced a simple filter to give greater preference to movies with more votes and higher ratings.
3. **Collaborative Filtering:** We used the powerful Surprise Library to build a collaborative filter based on single value decomposition. The RMSE obtained was less than 1 and the engine gave estimated ratings for a given user and movie.
4. **Hybrid Engine:** We brought together ideas from content and collaborative filterting to build an engine that gave movie suggestions to a particular user based on the estimated ratings that it had internally calculated for that user.

Previous -> [The Story of Film](https://www.kaggle.com/rounakbanik/the-story-of-film/)